In [ ]:

pip install Office365-REST-Python-Client

In [ ]:
#import packages
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
from pyspark.sql import SparkSession
import io
import pandas as pd
from pyspark.sql.functions import col

In [ ]:
#list files share point folder
def list_files_in_folder(ctx, folder_url):
    """List all files in a given SharePoint folder."""
    folder = ctx.web.get_folder_by_server_relative_url(folder_url)
    files = folder.files.get().execute_query()
    return [file.serverRelativeUrl for file in files]



In [ ]:
#read excel file as data frame from sharepoint site
def fetch_excel_to_dataframe(ctx, file_url, sheet_name,hdr):
    """Fetch Excel file from SharePoint and return as a pandas DataFrame."""
    file_stream = io.BytesIO()
    ctx.web.get_file_by_server_relative_url(file_url).download(file_stream).execute_query()
    print(f"File fetched successfully from: {file_url}")
    file_stream.seek(0)
    return pd.read_excel(file_stream,header=hdr,sheet_name=sheet_name, engine='openpyxl')


In [ ]:

#read CSV file as data frame from sharepoint site
def fetch_csv_to_dataframe(ctx, file_url, header=None):
    try:
        # Create an in-memory binary stream
        file_stream = io.BytesIO()
        # Download the file content into the binary stream
        ctx.web.get_file_by_server_relative_url(file_url).download(file_stream).execute_query()
        print(f"File fetched successfully from: {file_url}")
        # Reset the stream's position to the beginning
        file_stream.seek(0)
        # Read the content into a pandas DataFrame
        df = pd.read_csv(file_stream, header=header)
        return df
    
    except Exception as e:
        print(f"An error occurred: {e}")
        raise


In [ ]:
#write dataframe to sharepoint as csv file
def write_to_sharepoint(pandas_df,file_path,hdr=True):
    # Convert DataFrame to CSV
    csv_file_path = file_path
    pandas_df.to_csv(csv_file_path, index=False, header=hdr)

    # Upload CSV to SharePoint
    with open(csv_file_path, "rb") as file:
        target_url = f"{document_library}/{csv_file_path}"
        ctx.web.get_folder_by_server_relative_url(document_library).upload_file(csv_file_path, file.read()).execute_query()
    
    return 'written to sharepoint successfully'



In [ ]:

def pandas_to_lakehouse(pandas_df,lakehouse_path):
    spark_df = spark.createDataFrame(pandas_df)
    #lakehouse_path=f'abfss://************************@onelake.dfs.fabric.microsoft.com/62929a5b-b6b5-47bd-a9f9-4933e5c8a21b/Files/merged_file'
    spark_df.write.format("delta").mode("overwrite").save(lakehouse_path)

    return 'written to lakehouse'

In [ ]:
def spark_to_lakehouse(spark_df,lakehouse_path):
    spark_df = spark.createDataFrame(pandas_df)
    #lakehouse_path=f'abfss://************************@onelake.dfs.fabric.microsoft.com/62929a5b-b6b5-47bd-a9f9-4933e5c8a21b/Files/merged_file'
    spark_df.write.format("delta").mode("overwrite").save(lakehouse_path)
    return 'written to lakehouse'

In [ ]:
def convert_ctx_object(site_url,username,password):
    try:
     
        #site_url sample (https://**********.sharepoint.com/sites/fabricdemo)
        print(f"site_url: {site_url} (type: {type(site_url)})")  # Debugging
        ctx = ClientContext(site_url).with_credentials(UserCredential(username, password))
        return ctx
    except Exception as e:
        print(f"Error in authentication: {e}")
        return None